In [2]:
import numpy as np

#return the number k of the first k significant eigenvalues
#W is the sorted vector of the eigenvalues
def entropy(W):
    F = W
    for i,w in enumerate(W):
        F[i] = w[i]**2/(sum(x**2 for x in W))
    E = -1/np.log(len(W)) * sum(f * np.log(f) for f in F)
    f_sum = 0
    for i,f in enumerate(F):
        if f_sum > E:
            return i
        f_sum += f

